# TAREA 2:

1.- Hacer un modelo regresión logistica con woes con la tabla que generaron en la tarea 1.

2.- Hacer un árbol de clasificación con la misma tabla que el problema 1. No olviden que tienen que hacer el ajuste de hiperparámetros.

3.- Seleccionar con validación cruzada cuál es el mejor modelo de los dos problemas anteriores.

4.- Obtener los puntos óptimos de decisión para el modelo seleccionado.

En todos los casos obtener: Gini, KS, ROC en entrenamiento y validación.
Fecha de entrega: Viernes 7 de enero.


## 1.- Modelo de Regresión Logística con Woes

In [3]:
# Importamos las herramientas que utilizaremos

import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc
from pylab import*
import seaborn as sns

In [4]:
# Mandamos a llamar a la base que analizamos en la Tarea 1 esta es la que decidimos que tiene los 
# criterios necesarios y a la que se le hará la regresión. 

ruta = "C:/Users/cresp/Documents/GitHub/Seminario_9273/Tarea 2/"
base = "Base_clientes.csv"
bd = pd.read_csv(ruta+base)

In [5]:
bd.head()

,Unnamed: 0,num_caso,edad_credit,retrasos_max,comp_interno1,Otorgar
0,0,1,62.0,1.0,3039.513678,1.0
1,1,7,29.0,6.0,1547.112462,1.0
2,2,8,63.0,1.0,1215.805471,1.0
3,3,9,15.0,0.0,0.000000,1.0
4,4,10,16.0,1.0,0.000000,1.0


In [6]:
bd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     50000 non-null  int64  
 1   num_caso       50000 non-null  int64  
 2   edad_credit    50000 non-null  float64
 3   retrasos_max   50000 non-null  float64
 4   comp_interno1  50000 non-null  float64
 5   Otorgar        50000 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 2.3 MB


In [7]:
from optbinning import OptimalBinning

In [8]:
train, test = train_test_split(bd, test_size = 0.30, random_state = 100) 

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35000 entries, 48824 to 38408
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     35000 non-null  int64  
 1   num_caso       35000 non-null  int64  
 2   edad_credit    35000 non-null  float64
 3   retrasos_max   35000 non-null  float64
 4   comp_interno1  35000 non-null  float64
 5   Otorgar        35000 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 1.9 MB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 4253 to 25776
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     15000 non-null  int64  
 1   num_caso       15000 non-null  int64  
 2   edad_credit    15000 non-null  float64
 3   retrasos_max   15000 non-null  float64
 4   comp_interno1  15000 non-null  float64
 5   Otorgar        15000 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 820.3 KB


In [11]:
y = train.Otorgar

In [12]:
columnas = list(bd.columns.values)
print(columnas)

['Unnamed: 0', 'num_caso', 'edad_credit', 'retrasos_max', 'comp_interno1', 'Otorgar']


In [13]:
optb_num_caso = OptimalBinning(name="num_caso", dtype="numerical", solver="cp")
optb_edad_credit = OptimalBinning(name="edad_credit", dtype="categorical", solver="cp",)
optb_retrasos_max = OptimalBinning(name="retrasos_max", dtype="numerical", solver="cp")
optb_comp_interno1 = OptimalBinning(name="comp_interno1", dtype="numerical", solver="cp")
optb_Otorgar = OptimalBinning(name="Otorgar", dtype="categorical", solver="mip")

In [14]:
optb_num_caso.fit(train["num_caso"].values, y)
optb_edad_credit.fit(train["edad_credit"].values, y)
optb_retrasos_max.fit(train["retrasos_max"].values, y)
optb_comp_interno1.fit(train["comp_interno1"].values, y)
optb_Otorgar.fit(train["Otorgar"].values, y)

OptimalBinning(dtype='categorical', name='Otorgar', solver='mip')

A continuación se muestran las cotas superiores e inferiores de cada categoría o columna en las cuales podemos apreciar los rangos de cada una. 

In [15]:
display(optb_num_caso.splits)
display(optb_edad_credit.splits)
display(optb_retrasos_max.splits)
display(optb_comp_interno1.splits)
display(optb_Otorgar.splits)

array([55386.5, 66023.5, 72151.5, 78853.5, 84238. , 89131. , 94018. ])

[array([ 0., 12., 25., 14.]),
 array([13., 17., 15., 16.]),
 array([19., 23., 20.]),
 array([22., 24., 18., 26.]),
 array([11., 10., 21.]),
 array([ 9.,  3.,  8., 40., 28.,  5.]),
 array([ 6., 47., 48., 49., 50., 51., 52., 53., 54., 57., 56., 58., 59.,
        63., 62., 61., 46., 60., 55., 45., 32., 43.,  1.,  2.,  4.,  7.,
        27., 29., 30., 31., 44., 64., 34., 35., 36., 37., 38., 39., 41.,
        42., 33., 65.])]

array([0.5])

array([1265.4508667 , 1963.01922607, 2710.23303223])

[array([0., 1.])]

In [16]:
num_caso_table = optb_num_caso.binning_table
edad_credit_table = optb_edad_credit.binning_table
retrasos_max_table = optb_retrasos_max.binning_table
comp_interno1_table = optb_comp_interno1.binning_table
Otorgar_table = optb_Otorgar.binning_table

Ahora procedemos a mostrar las tablas que se crean de aplicar la construcción, estas tablas resultan de gran ayuda ya que con ellas calculamos los WoE's

In [17]:
display(num_caso_table.build())
display(edad_credit_table.build())
display(retrasos_max_table.build())
display(comp_interno1_table.build())
display(Otorgar_table.build())

,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 55386.50)",19589,0.559686,287,19302,0.985349,-0.374778,0.065998,0.008202
1,"[55386.50, 66023.50)",3744,0.106971,48,3696,0.987179,-0.510101,0.021989,0.002719
2,"[66023.50, 72151.50)",2220,0.063429,25,2195,0.988739,-0.641358,0.019454,0.002391
3,"[72151.50, 78853.50)",2285,0.065286,22,2263,0.990372,-0.7997,0.029082,0.003541
4,"[78853.50, 84238.00)",1909,0.054543,6,1903,0.996857,-1.925723,0.091376,0.009931
5,"[84238.00, 89131.00)",1750,0.050000,18,1732,0.989714,-0.732956,0.019251,0.002354
6,"[89131.00, 94018.00)",1750,0.050000,50,1700,0.971429,0.307343,0.005487,0.000683
7,"[94018.00, inf)",1753,0.050086,285,1468,0.837422,2.194537,0.750017,0.078554
8,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
9,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"[0.0, 12.0, 25.0, 14.0]",2287,0.065343,484,1803,0.788369,2.518582,1.512516,0.150990
1,"[13.0, 17.0, 15.0, 16.0]",2744,0.078400,101,2643,0.963192,0.569155,0.033668,0.004153
2,"[19.0, 23.0, 20.0]",1895,0.054143,50,1845,0.973615,0.225492,0.003072,0.000383
3,"[22.0, 24.0, 18.0, 26.0]",2549,0.072829,57,2492,0.977638,0.055914,0.000234,0.000029
4,"[11.0, 10.0, 21.0]",1937,0.055343,38,1899,0.980382,-0.077793,0.000323,0.000040
5,"[9.0, 3.0, 8.0, 40.0, 28.0, 5.0]",3637,0.103914,10,3627,0.997250,-2.059872,0.190280,0.020309
6,"[6.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0...",19951,0.570029,1,19950,0.999950,-6.06728,3.524964,0.197518
7,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
8,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,35000,1.000000,741,34259,0.978829,,5.265057,0.373422


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 0.50)",7416,0.211886,404,7012,0.945523,0.979741,0.333634,0.040112
1,"[0.50, inf)",27584,0.788114,337,27247,0.987783,-0.558912,0.190328,0.023486
2,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
3,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,35000,1.000000,741,34259,0.978829,,0.523962,0.063599


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"(-inf, 1265.45)",28109,0.803114,630,27479,0.977587,0.058246,0.002802,0.000350
1,"[1265.45, 1963.02)",1961,0.056029,51,1910,0.973993,0.210671,0.002754,0.000344
2,"[1963.02, 2710.23)",1776,0.050743,59,1717,0.966779,0.462908,0.013658,0.001692
3,"[2710.23, inf)",3154,0.090114,1,3153,0.999683,-4.222406,0.382908,0.028836
4,Special,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
5,Missing,0,0.000000,0,0,0.000000,0.0,0.000000,0.000000
Totals,,35000,1.000000,741,34259,0.978829,,0.402121,0.031222


,Bin,Count,Count (%),Non-event,Event,Event rate,WoE,IV,JS
0,"[0.0, 1.0]",35000,1.0,741,34259,0.978829,0.0,0.0,0.0
1,Special,0,0.0,0,0,0.000000,0.0,0.0,0.0
2,Missing,0,0.0,0,0,0.000000,0.0,0.0,0.0
Totals,,35000,1.0,741,34259,0.978829,,0.0,0.0


In [20]:
binning_table = optb.binning_table

NameError: name 'optb' is not defined

In [21]:
binning_table.build()

NameError: name 'binning_table' is not defined